In [1]:
# use avik_2 environment with sklearn==0.23.2 version to run
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [2]:
from itertools import chain
from tqdm.auto import tqdm
#import nltk
import sklearn
import scipy.stats
import random
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

from seqeval.metrics import classification_report

In [3]:
import pandas as pd

In [4]:
# For Human Induced
dataset_path = "~/sr_drive/Avik/Annotations/results/finalized_dataset_human_induced/"
train_data = pd.read_csv(dataset_path + "train.csv")
dev_data = pd.read_csv(dataset_path + "dev.csv")
# test_data = pd.read_csv(dataset_path + "test.csv")
test_data = pd.read_csv("~/sr_drive/Avik/Annotations/results/finalized_dataset/human_auto_test_dataset.csv")

train_sents = []
valid_sents = []
test_sents = []

train_data = train_data.fillna("None")
dev_data = dev_data.fillna("None")
test_data = test_data.fillna("None")

for bug_id, group_df in tqdm(train_data.groupby('Bug Id')):
    train_entry = []
    for row_num, row in group_df.iterrows():
        train_entry.append((row['Entity'], row['POS Tag'], row['BIO Tag']))
    train_sents.append(train_entry)
for bug_id, group_df in tqdm(dev_data.groupby('Bug Id')):
    dev_entry = []
    for row_num, row in group_df.iterrows():
        dev_entry.append((row['Entity'], row['POS Tag'], row['BIO Tag']))
    valid_sents.append(dev_entry)
for bug_id, group_df in tqdm(test_data.groupby('Bug Id')):
    test_entry = []
    for row_num, row in group_df.iterrows():
        test_entry.append((row['Entity'], row['POS Tag'], row['BIO Tag']))
    test_sents.append(test_entry)
    
bio_tagging_labels = train_data['BIO Tag'].unique()

  0%|          | 0/64787 [00:00<?, ?it/s]

  0%|          | 0/99 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

In [4]:
# for Launchpad
dataset_path = "~/sr_drive/Avik/Annotations/results/finalized_dataset_human_newsplit/"
train_data = pd.read_csv(dataset_path + "train.csv")
dev_data = pd.read_csv(dataset_path + "dev.csv")
test_data = pd.read_csv("/home/somnath-am/sr_drive/Avik/Launchpad/launchpad_bio_tagging_human.csv")

train_sents = []
valid_sents = []
test_sents = []

train_data = train_data.fillna("None")
dev_data = dev_data.fillna("None")
test_data = test_data.fillna("None")

for bug_id, group_df in tqdm(train_data.groupby('Bug Id')):
    train_entry = []
    for row_num, row in group_df.iterrows():
        train_entry.append((row['Entity'], row['POS Tag'], row['BIO Tag']))
    train_sents.append(train_entry)
for bug_id, group_df in tqdm(dev_data.groupby('Bug Id')):
    dev_entry = []
    for row_num, row in group_df.iterrows():
        dev_entry.append((row['Entity'], row['POS Tag'], row['BIO Tag']))
    valid_sents.append(dev_entry)
for bug_id, group_df in tqdm(test_data.groupby('Bug Id')):
    test_entry = []
    for row_num, row in group_df.iterrows():
        test_entry.append((row['Entity'], row['POS Tag'], row['BIO Tag']))
    test_sents.append(test_entry)
    
bio_tagging_labels = train_data['BIO Tag'].unique()

  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/151 [00:00<?, ?it/s]

In [ ]:
dataset_path = "~/sr_drive/Avik/Annotations/results/finalized_dataset/"
bio_tagging_df = pd.read_csv(dataset_path + "updated_merged_BIO_tagging_removing_verb_adjacent_pos_tags.csv")
test_data = pd.read_csv(dataset_path + "human_test_dataset.csv")

In [ ]:
test_data = pd.read_csv("~/sr_drive/Avik/Launchpad/launchpad_bio_tagging.csv")

In [ ]:
bio_tagging_labels = bio_tagging_df['BIO Tag'].unique()

In [ ]:
train_yrs = list(range(2004, 2014))
valid_yrs = list(range(2014, 2016))
test_yrs = list(range(2016, 2020))

In [5]:
# test data support check
test_data[(test_data['BIO Tag'] != 'O')]['BIO Tag'].value_counts()

B-Package               696
B-Software_Component    608
B-OS                    595
B-Peripheral            524
B-Command               410
I-Peripheral            347
I-Error                 232
B-Architecture          223
I-OS                    208
B-Organization          206
I-Software_Component    125
I-Organization          119
I-Command               118
B-Error                  88
B-Extension              87
I-Package                50
I-Architecture            3
Name: BIO Tag, dtype: int64

In [ ]:
# ## Do not run this
# arr = test_data['Bug Id'].unique()
# sampled_ids = np.random.choice(arr, size=round(len(arr)*0.4), replace=False)
# bug_id_to_sents = {}
# for idx in tqdm(range(bio_tagging_df.shape[0])):
#     if bio_tagging_df['Bug Id'][idx] not in bug_id_to_sents:
#         bug_id_to_sents[bio_tagging_df['Bug Id'][idx]] = []
#     bug_id_to_sents[bio_tagging_df['Bug Id'][idx]].append((str(bio_tagging_df['Entity'][idx]), str(bio_tagging_df['POS Tag'][idx]), bio_tagging_df['BIO Tag'][idx], bio_tagging_df['year'][idx]))
# train_sents = []
# valid_sents = []
# for bug_id, entry in tqdm(bug_id_to_sents.items()):
#     if entry[0][-1] in train_yrs:
#         train_sents.append([x[:-1] for x in entry])
#     elif entry[0][-1] in valid_yrs:
#         valid_sents.append([x[:-1] for x in entry])
# test_sents = []
# test_data = test_data.fillna("None")
# for bug_id, group_df in tqdm(test_data.groupby('Bug Id')):
#     test_entry = []
#     valid_entry = []
#     if bug_id in sampled_ids:
#         for row_num, row in group_df.iterrows():
#             valid_entry.append((row['Entity'], row['POS Tag'], row['BIO Tag']))
#         valid_sents.append(valid_entry)
#     else:
#         for row_num, row in group_df.iterrows():
#             test_entry.append((row['Entity'], row['POS Tag'], row['BIO Tag']))
#         test_sents.append(test_entry)

In [ ]:
bug_id_to_sents = {}
for idx in tqdm(range(bio_tagging_df.shape[0])):
    if bio_tagging_df['Bug Id'][idx] not in bug_id_to_sents:
        bug_id_to_sents[bio_tagging_df['Bug Id'][idx]] = []
    bug_id_to_sents[bio_tagging_df['Bug Id'][idx]].append((str(bio_tagging_df['Entity'][idx]), str(bio_tagging_df['POS Tag'][idx]), bio_tagging_df['BIO Tag'][idx], bio_tagging_df['year'][idx]))
train_sents = []
valid_sents = []
for bug_id, entry in tqdm(bug_id_to_sents.items()):
    if entry[0][-1] in train_yrs:
        train_sents.append([x[:-1] for x in entry])
    elif entry[0][-1] in valid_yrs:
        valid_sents.append([x[:-1] for x in entry])
#     else:
#         test_sents.append([x[:-1] for x in entry])




In [ ]:
test_sents = []
test_data = test_data.fillna("None")
for bug_id, group_df in tqdm(test_data.groupby('Bug Id')):
    test_entry = []
    for row_num, row in group_df.iterrows():
        test_entry.append((row['Entity'], row['POS Tag'], row['BIO Tag']))
    test_sents.append(test_entry)

In [6]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [7]:
# from sklearn.model_selection import train_test_split
# train_sents, test_sents = train_test_split(sents, test_size=0.2, random_state=1)

# train_sents, valid_sents = train_test_split(train_sents, test_size=0.25, random_state=1)
# print(len(train_sents), len(valid_sents), len(test_sents), len(sents))

In [8]:
%%time
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_valid = [sent2features(s) for s in valid_sents]
y_valid = [sent2labels(s) for s in valid_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

CPU times: user 24.4 s, sys: 4.19 s, total: 28.6 s
Wall time: 28.6 s


In [9]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np

def print_metrics(fout, y_label, y_pred):
    y_label = list(np.concatenate(y_label).flat)
    y_pred = list(np.concatenate(y_pred).flat)
    fout.write(f'Accuracy score: {accuracy_score(y_label, y_pred)}\n')

def print_f1_score(fout, y_label, y_pred):
    global bio_tagging_labels
    y_label = list(np.concatenate(y_label).flat)
    y_pred = list(np.concatenate(y_pred).flat)
    labels = bio_tagging_labels
#     labels=['B-Command', 'B-Error_description', 'B-Error_name', 'B-Extension',
#        'B-OS', 'B-Package', 'B-URL', 'B-Ubuntu_distribution',
#        'I-Error_description', 'I-Extension', 'I-URL', 'O']
    classwise_f1_score = f1_score(y_label, y_pred, average=None, labels=labels)
    fout.write('Classwise F-1 Score\n')
    for label,score in zip(labels, classwise_f1_score):
        fout.write(str(label + ' : ' + str(score) + '\n'))
    fout.write('\n')

In [12]:
log_fout = open('CRF_logs.txt', 'w')

In [10]:
%%time
crf1 = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=20,
    all_possible_transitions=False,
)
crf1.fit(X_train, y_train)
print('Fit Done')
log_fout.write("Training for algorithm 'lbfgs' done\n")

with open('LinearCRF_metrics.txt', 'a') as fout:
    y_valid_pred = crf1.predict(X_valid)
    fout.write(f"Algorithm: 'lbfgs'\n")
    print_metrics(fout, y_valid, y_valid_pred)
    print_f1_score(fout, y_valid, y_valid_pred)

Fit Done


NameError: name 'log_fout' is not defined

In [ ]:
%%time
crf2 = sklearn_crfsuite.CRF(
    algorithm='l2sgd',
    c2=0.1,
    max_iterations=20,
    all_possible_transitions=False,
)
crf2.fit(X_train, y_train)
print('Fit Done')
log_fout.write("Training for algorithm 'l2sgd' done\n")

with open('LinearCRF_metrics.txt', 'a') as fout:
    y_valid_pred = crf2.predict(X_valid)
    fout.write(f"Algorithm: 'l2sgd'\n")
    print_metrics(fout, y_valid, y_valid_pred)
    print_f1_score(fout, y_valid, y_valid_pred)

In [ ]:
%%time
crf3 = sklearn_crfsuite.CRF(
    algorithm='ap',
    max_iterations=20,
    all_possible_transitions=False,
)
crf3.fit(X_train, y_train)
print('Fit Done')
log_fout.write("Training for algorithm 'ap' done\n")

with open('LinearCRF_metrics.txt', 'a') as fout:
    y_valid_pred = crf3.predict(X_valid)
    fout.write(f"Algorithm: 'ap'\n")
    print_metrics(fout, y_valid, y_valid_pred)
    print_f1_score(fout, y_valid, y_valid_pred)

In [ ]:
%%time
crf4 = sklearn_crfsuite.CRF(
    algorithm='pa',
    max_iterations=20,
    all_possible_transitions=False,
)
crf4.fit(X_train, y_train)
print('Fit Done')
log_fout.write("Training for algorithm 'pa' done\n")

with open('LinearCRF_metrics.txt', 'a') as fout:
    y_valid_pred = crf4.predict(X_valid)
    fout.write(f"Algorithm: 'pa'\n")
    print_metrics(fout, y_valid, y_valid_pred)
    print_f1_score(fout, y_valid, y_valid_pred)

In [ ]:
%%time
crf5 = sklearn_crfsuite.CRF(
    algorithm='arow',
    max_iterations=20,
    all_possible_transitions=False,
)
crf5.fit(X_train, y_train)
print('Fit Done')
log_fout.write("Training for algorithm 'arow' done\n")

with open('LinearCRF_metrics.txt', 'a') as fout:
    y_valid_pred = crf5.predict(X_valid)
    fout.write(f"Algorithm: 'arow'\n")
    print_metrics(fout, y_valid, y_valid_pred)
    print_f1_score(fout, y_valid, y_valid_pred)

In [ ]:
%%time
crf6 = sklearn_crfsuite.CRF(
    algorithm='pa',
    max_iterations=40,
    all_possible_transitions=False,
)
crf6.fit(X_train, y_train)
print('Fit Done')
log_fout.write("Training for algorithm 'pa', # of iterations = 40\n")

with open('LinearCRF_metrics.txt', 'a') as fout:
    y_valid_pred = crf6.predict(X_valid)
    fout.write(f"Algorithm: 'pa', # of iterations = 40\n")
    print_metrics(fout, y_valid, y_valid_pred)
    print_f1_score(fout, y_valid, y_valid_pred)

In [ ]:
%%time
crf7 = sklearn_crfsuite.CRF(
    algorithm='pa',
    max_iterations=60,
    all_possible_transitions=False,
)
crf7.fit(X_train, y_train)
print('Fit Done')
log_fout.write("Training for algorithm 'pa', # of iterations = 60\n")

with open('LinearCRF_metrics.txt', 'a') as fout:
    y_valid_pred = crf7.predict(X_valid)
    fout.write(f"Algorithm: 'pa', # of iterations = 60\n")
    print_metrics(fout, y_valid, y_valid_pred)
    print_f1_score(fout, y_valid, y_valid_pred)

In [ ]:
%%time
crf8 = sklearn_crfsuite.CRF(
    algorithm='pa',
    max_iterations=100,
    all_possible_transitions=False,
)
crf8.fit(X_train, y_train)
print('Fit Done')
log_fout.write("Training for algorithm 'pa', # of iterations = 100\n")

with open('LinearCRF_metrics.txt', 'a') as fout:
    y_valid_pred = crf8.predict(X_valid)
    fout.write(f"Algorithm: 'pa', # of iterations = 100\n")
    print_metrics(fout, y_valid, y_valid_pred)
    print_f1_score(fout, y_valid, y_valid_pred)

In [13]:
%%time
crf9 = sklearn_crfsuite.CRF(
    algorithm='pa',
    max_iterations=150,
    all_possible_transitions=False,
)
crf9.fit(X_train, y_train)
print('Fit Done')
log_fout.write("Training for algorithm 'pa', # of iterations = 150\n")

with open('LinearCRF_metrics.txt', 'a') as fout:
    y_valid_pred = crf9.predict(X_valid)
    fout.write(f"Algorithm: 'pa', # of iterations = 150\n")
    print_metrics(fout, y_valid, y_valid_pred)
    print_f1_score(fout, y_valid, y_valid_pred)

Fit Done


TypeError: f1_score() got an unexpected keyword argument 'labels'

In [ ]:
with open('LinearCRF_metrics.txt', 'a') as fout:
    y_test_pred = crf9.predict(X_test)
    fout.write(f"Algorithm: 'pa', # of iterations = 150, on test set\n")
    print_metrics(fout, y_test, y_test_pred)
    print_f1_score(fout, y_valid, y_valid_pred)

In [ ]:
# import eli5
# eli5.show_weights(crf, top=30)

In [ ]:
import sklearn
sklearn.__version__

In [14]:
from sklearn_crfsuite.metrics import flat_classification_report
y_pred = crf9.predict(X_test)
labels = list(crf9.classes_)
print(labels)
labels.remove('O') # remove 'O' label from evaluation
sorted_labels = sorted(labels,key=lambda name: (name[2:], name[0])) # group B and I results
print(flat_classification_report(y_test, y_pred, labels=sorted_labels, digits=3))

['O', 'B-Package', 'B-Extension', 'B-Organization', 'B-Command', 'B-Software_Component', 'B-OS', 'B-Peripheral', 'I-Software_Component', 'B-Error', 'I-Package', 'B-Architecture', 'I-Organization', 'I-Error', 'I-Peripheral', 'I-Extension', 'I-OS', 'I-Architecture', 'I-Command']


/home/somnath-am/anaconda3/envs/avik_2/lib/python3.9/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass labels=['B-Architecture', 'I-Architecture', 'B-Command', 'I-Command', 'B-Error', 'I-Error', 'B-Extension', 'I-Extension', 'B-OS', 'I-OS', 'B-Organization', 'I-Organization', 'B-Package', 'I-Package', 'B-Peripheral', 'I-Peripheral', 'B-Software_Component', 'I-Software_Component'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


                      precision    recall  f1-score   support

      B-Architecture      0.962     0.951     0.956      1603
      I-Architecture      0.800     0.320     0.457        25
           B-Command      0.981     0.833     0.901      2189
           I-Command      0.944     0.058     0.110       291
             B-Error      0.950     0.722     0.820       756
             I-Error      0.987     0.385     0.554      1582
         B-Extension      0.973     0.966     0.970      1073
         I-Extension      0.929     0.897     0.912        29
                B-OS      0.910     0.916     0.913      2594
                I-OS      0.744     0.480     0.584       641
      B-Organization      0.905     0.825     0.863      1475
      I-Organization      0.887     0.558     0.685       509
           B-Package      0.928     0.892     0.909      7547
           I-Package      0.893     0.118     0.209       211
        B-Peripheral      0.949     0.848     0.896      1612
       

In [15]:
from seqeval.metrics import classification_report, precision_score, recall_score, f1_score
y_pred = crf9.predict(X_test)
labels = list(crf9.classes_)
labels.remove('O') # remove 'O' label from evaluation
sorted_labels = sorted(labels,key=lambda name: (name[2:], name[0])) # group B and I results
print(classification_report(y_test, y_pred))#, labels=sorted_labels, digits=3))
print("Precision score : {}".format(precision_score(y_test, y_pred)))
print("Recall score : {}".format(recall_score(y_test, y_pred)))
print("f1-score score : {}".format(f1_score(y_test, y_pred)))

                    precision    recall  f1-score   support

      Architecture       0.96      0.95      0.96      1603
           Command       0.98      0.83      0.90      2189
             Error       0.92      0.70      0.79       781
         Extension       0.97      0.97      0.97      1073
                OS       0.78      0.79      0.78      2594
      Organization       0.82      0.75      0.78      1475
           Package       0.93      0.89      0.91      7547
        Peripheral       0.93      0.83      0.88      1612
Software_Component       0.97      0.89      0.92      2392

         micro avg       0.91      0.86      0.89     21266
         macro avg       0.92      0.84      0.88     21266
      weighted avg       0.92      0.86      0.89     21266

Precision score : 0.9148541645785306
Recall score : 0.858412489419731
f1-score score : 0.8857350800582241


In [10]:
## This code segment helps in finding out the accuracy of the model on the human data
classwise_acc_dict = {}
classwise_count_dict = {}
for i in tqdm(range(len(y_test))):
    for j in range(len(y_test[i])):
        if y_test[i][j] != 'O':
            ent = y_test[i][j][2:]
            
            if ent not in classwise_count_dict:
                classwise_count_dict[ent] = 0
            classwise_count_dict[ent] = classwise_count_dict[ent] + 1
            
            if ent not in classwise_acc_dict:
                classwise_acc_dict[ent] = 0
            if y_pred[i][j][2:] == ent:
                classwise_acc_dict[ent] = classwise_acc_dict[ent] + 1
sum_correct = 0
for key in classwise_acc_dict.keys():
    sum_correct += classwise_acc_dict[key]
    classwise_acc_dict[key] = classwise_acc_dict[key]/classwise_count_dict[key]
print("The classwise accuracy : ")
print(classwise_acc_dict)
overall_accuracy = sum_correct/sum(classwise_count_dict.values())
print(f"The overall accuracy : {overall_accuracy}")

  0%|          | 0/351 [00:00<?, ?it/s]

The classwise accuracy : 
{'Command': 0.05333333333333334, 'OS': 0.6764705882352942, 'Package': 0.7366003062787136, 'Architecture': 0.959915611814346, 'Organization': 0.5384615384615384, 'Peripheral': 0.14691151919866444, 'Error': 0.1509433962264151, 'Extension': 0.6296296296296297, 'Software_Component': 0.1201923076923077}
The overall accuracy : 0.44327815906338314


In [ ]:
## Finding the performance of model, to see if it is able to mark the entities marked by human
def accuracy_of_model(y_test, y_pred):
    

In [ ]:
def print_f1_score(y_label, y_pred):
    y_label = list(np.concatenate(y_label).flat)
    y_pred = list(np.concatenate(y_pred).flat)
    print('Micro f1 score: ', f1_score(y_label, y_pred, average='micro'))
    print('Macro f1 score: ', f1_score(y_label, y_pred, average='macro'))
    print('Weighted f1 score: ', f1_score(y_label, y_pred, average='weighted'))

In [ ]:
y_valid_pred = crf1.predict(X_valid)
print_f1_score(y_valid, y_valid_pred)

In [ ]:
y_valid_pred = crf2.predict(X_valid)
print_f1_score(y_valid, y_valid_pred)

In [ ]:
y_valid_pred = crf3.predict(X_valid)
print_f1_score(y_valid, y_valid_pred)

In [ ]:
y_valid_pred = crf4.predict(X_valid)
print_f1_score(y_valid, y_valid_pred)

In [ ]:
y_valid_pred = crf5.predict(X_valid)
print_f1_score(y_valid, y_valid_pred)

In [ ]:
y_valid_pred = crf6.predict(X_valid)
print_f1_score(y_valid, y_valid_pred)

In [ ]:
y_valid_pred = crf7.predict(X_valid)
print_f1_score(y_valid, y_valid_pred)

In [ ]:
y_valid_pred = crf8.predict(X_valid)
print_f1_score(y_valid, y_valid_pred)

In [ ]:
y_valid_pred = crf9.predict(X_valid)
print_f1_score(y_valid, y_valid_pred)

In [ ]:
import torch
torch.save(crf9, "Linear_CRF_human_induced.pth")

In [ ]:
crf9 = torch.load("Linear_CRF_model.pth")